A very quick example of pulling out unique values from one field and writing them as a CSV to a local file.

In [11]:
from pyspark.sql.functions import col

In [24]:
# You can add "-100k" or "-1M" before the "." to load smaller data sets for testing
idb_df = sqlContext.read.parquet("/guoda/data/idigbio-20190426T013311.parquet")

In [13]:
idb_df.count()

115173471

In [14]:
idb_df.printSchema()

root
 |-- barcodevalue: string (nullable = true)
 |-- basisofrecord: string (nullable = true)
 |-- bed: string (nullable = true)
 |-- canonicalname: string (nullable = true)
 |-- catalognumber: string (nullable = true)
 |-- class: string (nullable = true)
 |-- collectioncode: string (nullable = true)
 |-- collectionid: string (nullable = true)
 |-- collectionname: string (nullable = true)
 |-- collector: string (nullable = true)
 |-- commonname: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- coordinateuncertainty: float (nullable = true)
 |-- country: string (nullable = true)
 |-- countrycode: string (nullable = true)
 |-- county: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- coreid: string (nullable = true)
 |    |-- dc:rights: string (nullable = true)
 |    |-- dcterms:accessRights: string (nullable = true)
 |    |-- dcterms:bibliographicCitation: string (nullable = true)
 |    |-- dcterms:language: string (nullable = true)
 |    |-- d

In [25]:
distinct_habitats = (idb_df
                     .groupBy(col("data.dwc:habitat"))
                     .count()
                     )

In [23]:
# Uncomment for testing, otherwise leave commented as it causes extra gathering work
# in addition to the writing of the CSV below
distinct_habitats.show(10, truncate=False)

+--------------------------------------------------------+-----+
|dwc:habitat                                             |count|
+--------------------------------------------------------+-----+
|disturbed thickets and fields near watercourses to ocean|1    |
|epiphyte on fallen tree                                 |1    |
|dry oamy soil                                           |1    |
|epiphyte in monte                                       |1    |
|monte                                                   |4    |
|Bosque primario, suelo de arena blanca                  |1    |
|disturbed forest & trailside                            |1    |
|Floodplain.; Compact, dry soil.                         |1    |
|In clay soil on top of sharp ridge                      |1    |
|Beitemark. Blokk/jord. På jord på blokker i beitemark   |1    |
+--------------------------------------------------------+-----+
only showing top 10 rows



In [26]:
# toPandas is inefficient (15 minutes) but simpler than working with HDFS
(distinct_habitats
 .toPandas()
 .to_csv("distinct_habitats.csv", index=False, header=True)
 )